In [15]:
# Test  AI Agent framework  AI A real estate agent 

In [156]:
import os , json, time , gc
import logging


from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
from openai.types.chat import (ChatCompletion, 
                               ChatCompletionChunk,
                               ChatCompletionContentPartTextParam, 
                               ChatCompletionContentPartImageParam,
                               ChatCompletionStreamOptionsParam)
import asyncio
import aiohttp
import pandas as pd
import re
import torch


import nest_asyncio
nest_asyncio.apply() # for jupyter notebook

In [157]:
# import crewai AI agent framework # pip install crewai
from crewai import Crew, Agent, Task, Process 

from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai.tools import BaseTool, tool
from crewai import LLM
from pydantic import BaseModel, Field
from typing import List, Dict, Type
from crewai.project import CrewBase, agent, task, crew

import crewai
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any


In [158]:
from crawl4ai import  AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode
import crawl4ai


from crawl4ai.chunking_strategy import (RegexChunking, 
                                        NlpSentenceChunking,
                                        OverlappingWindowChunking, 
                                        SlidingWindowChunking,
                                        FixedLengthWordChunking, 
                                        TopicSegmentationChunking) 

from crawl4ai.extraction_strategy import ( JsonCssExtractionStrategy, 
                                           JsonXPathExtractionStrategy,
                                          LLMExtractionStrategy, 
                                          CosineStrategy)
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator


from crawl4ai.content_filter_strategy import BM25ContentFilter, PruningContentFilter, LLMContentFilter

In [159]:
class CFG:
    # search engine
    num_results = 3 # 5
    queryTime = "month" #"year"

    # Azure Opena Model
    llmModel1 = "gpt-4o-mini"
    llmModel2 = "gpt-4o"
    embedModel1 = "text-embedding-3-small" #1528 openai embedding model
    embedModel2 = "text-embedding-3-large"  # around 3000 dimension
    embedModel3 = "text-embedding-ada-002"
    embedModel4 = "azure-text-embedding-3-small"
    embedModel5 = "azure-text-embedding-3-large"

  
    
    openEmbedModel1 = "intfloat/multilingual-e5-small" # 512 dimension
    openEmbedModel2 = "sentence-transformers/distiluse-base-multilingual-cased-v1"
    openEmbedModel3 = "intfloat/multilingual-e5-large" # curent  # 1024 dimension 
    openEmbedModel4 = "Alibaba-NLP/gte-multilingual-base"
    openEmbedModel5 = "nomic-embed-text" # 768 dimension , support 8196 input 


    visionEmbedModel1 = "nomic-ai/nomic-embed-vision-v1.5"
    visionEmbedModel2 =  "openai/clip-vit-base-patch16" # openai vision embeding model 


    promptInjectModel1 = "protectai/deberta-v3-base-prompt-injection-v2"

    htmlMarkdownModel1 = "jinaai/reader-lm-0.5b"
    htmlMarkdownModel2 = "jinaai/reader-lm-1.5b"


    # Load locally Open source Embedding model 
    modelPath = "./model" 
    fullModelPath = modelPath + "/" + openEmbedModel3

In [160]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [161]:
logger= logging.getLogger(__name__)

In [162]:
def set_log_level(logger ,level: str):
    if level == "DEBUG":
        logger.setLevel(logging.DEBUG)
    elif level == "INFO":
        logger.setLevel(logging.INFO)
    elif level == "WARNING":
        logger.setLevel(logging.WARNING)
    elif level == "ERROR":
        logger.setLevel(logging.ERROR)
    elif level == "CRITICAL":
        logger.setLevel(logging.CRITICAL)
    else:
        logger.setLevel(logging.DEBUG)

In [163]:
set_log_level(logger, "DEBUG")

In [164]:
from langchain_ollama import OllamaLLM, ChatOllama
from langchain_openai import ChatOpenAI

In [165]:
import ctypes
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        gc.collect()
        time.sleep(0.3)

In [166]:
query="Find me a 3 bedroom house in Hong Kong under $5 million"

In [167]:
# from crewai_tools import BraveSearchTool

# # Initialize the tool for internet searching capabilities
# tool = BraveSearchTool()

# SearxNG Web Search Engine

You can [check this link](https://docs.searxng.org/dev/search_api.html) for more informations about `Searx API` parameters.
#### Download SearxNG docker for local host setup web search Engine

In [168]:
import pprint
from langchain_community.utilities import SearxSearchWrapper

In [169]:
searchGeneralConfig = "!general"
searchImageConfig = "!images !bii !brimg !ddi !goi"
searchVideoConfig = "!videos !biv !brvid !ddv !gov"
searchNewConfig = "!news !gon !yhn"
searchMapConfig = "!map paris"
searchYouTubeConfig = "!yt"
searchSocialMedia = "!social_media !re !toot !mah !leco"

In [170]:
searxHost = "http://localhost:8080" #for langchain
searxHost2 = "http://localhost:8080/search" # for Searxng API directly
search = SearxSearchWrapper(searx_host=searxHost, k=5)

In [171]:
## Find URL web page for the query

def searchURL(q, k=3, qureyTime=CFG.queryTime, 
              engines=["duckduckgo", "qwant", "google", "brave"],
              categories = "general",
              querySuffix = None):
    results = search.results(q, 
                        num_results=k,
                        time_range = qureyTime,
                        engines= engines,
                        categories = categories,
                       query_suffix = querySuffix,
                       )
    
    return results

async def asyncSearchURL(q, k=3 , qureyTime=CFG.queryTime, 
                         engines=["duckduckgo", "qwant", "google", "brave"],
                         categories = "general",
                         querySuffix = None
                        ):
    startTime = time.time()
    results = await search.aresults(q, 
                        num_results=k,
                        time_range = qureyTime,
                        engines= engines,
                        categories = categories,
                        query_suffix = querySuffix,)
    print(f"Time take: {time.time() - startTime}")
    print(f"Query Categores : {categories}")
    return results
    

In [172]:
# %%time
queryTime = "month" #"year"#"month" #"year" #"day"
k = 5#5
currentQuery = "What is Hong Kong Weather Now?" #q2 #"Hong Kong Kwun Tong" #q3#tempQuery#q1
categories =  []#[""] #["science"]#["science", "image"]
query_suffix = None #searchMapConfig   # searchVideoConfig
urls = await asyncSearchURL(currentQuery, k=k, qureyTime=queryTime, categories=categories,  querySuffix= query_suffix)
print(currentQuery)
urls

Time take: 0.8830294609069824
Query Categores : []
What is Hong Kong Weather Now?


[{'snippet': '5 hours ago — At 1 p.m. at Hong Kong Observatory : Air Temperature : 29 degrees Celsius Relative Humidity : 75 per cent Weather Cartoon : No.',
  'title': 'Current Weather',
  'link': 'https://www.hko.gov.hk/textonly/v2/forecast/englishwx2.htm',
  'engines': ['google'],
  'category': 'general'},
 {'snippet': '2 weeks ago - Conflicting weather models have sparked speculation about a potential tropical cyclone approaching Hong Kong next week. However, meteorological experts are cautioning that predictions beyond five days remain highly uncertain.',
  'title': 'HK typhoon forecast: too early to tell, experts warn',
  'link': 'https://www.thestandard.com.hk/hong-kong-news/article/302919/HK-typhoon-forecast-too-early-to-tell-experts-warn',
  'engines': ['brave'],
  'category': 'general'},
 {'snippet': 'The temperature in Hong Kong today in the early morning is 25 ° C. If you take into account factors such as wind, humidity and other weather conditions, the temperatures can fee

In [173]:
df = pd.DataFrame(urls)
df

,snippet,title,link,engines,category
0,5 hours ago — At 1 p.m. at Hong Kong Observato...,Current Weather,https://www.hko.gov.hk/textonly/v2/forecast/en...,[google],general
1,2 weeks ago - Conflicting weather models have ...,"HK typhoon forecast: too early to tell, expert...",https://www.thestandard.com.hk/hong-kong-news/...,[brave],general
2,The temperature in Hong Kong today in the earl...,"Hong Kong local weather (live): today, hourly ...",https://www.weather25.com/asia/hong-kong?page=...,[duckduckgo],general
3,4 days ago — At 10 p.m. at Hong Kong Observato...,Current Weather,https://www.hko.gov.hk/textonly/forecast/engli...,[google],general
4,1 week ago - Summer in Hong Kong is from late ...,Best Times to Visit Hong Kong 2025 & Typhoon S...,https://www.chinahighlights.com/hong-kong/weat...,[brave],general


# Web scaping tool used Crawl4AI tool base on Playwrigth and Chrowmimum browswer


In [174]:
# Bros
# Test  playwright engine
from playwright.async_api import async_playwright

async def playwrightTestWeb():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True) # launch browser
        page = await browser.new_page()
        await page.goto('https://www.google.com')
        print(f"Title of the page: {await page.title()}")
        await page.screenshot(path='example.png')# save screenshot
        await browser.close()

In [175]:
await playwrightTestWeb() # test the browser pylaywright engine

Title of the page: Google


In [179]:
browser_config= BrowserConfig(
    browser_type="chromium",
    headless=True,
    verbose=True,
    # for better performance in Docker or low memory menivroment 
    extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

In [187]:
crawler_config1 = CrawlerRunConfig(
            # fetch_ssl_certificate=True,
            word_count_threshold=10, 
            excluded_tags=["form", "header"],
            exclude_external_links=True,
    
            markdown_generator=DefaultMarkdownGenerator(),
            # Content processing
            process_iframes=True,
            remove_overlay_elements=True,

            # Cache control 
            # cache_mode=CacheMode.ENABLED,
            cache_mode=CacheMode.BYPASS, # fetch each time
    
            page_timeout=120000, #60000, #30000, # for each image 
            wait_for_images=True,
            verbose=True,
)

In [ ]:
async def getWebPageContent(url):
    """
    Fetch the single content of a web page using AsyncWebCrawler.
    """
    global browser_config
    async with AsyncWebCrawler() as crawler:
        # Configure the crawler
        result = await crawler.arun(
            url=url,
            browser_config=browser_config,
            run_config=CrawlerRunConfig(
                cache_mode=CacheMode.BYPASS,  # Disable caching
                # max_depth=1,  # Limit the depth of crawling
            )
        )
        return result
    

In [206]:
from crawl4ai.async_dispatcher import MemoryAdaptiveDispatcher
from crawl4ai import CrawlerMonitor, DisplayMode

async def getWebPageContentsConcurrent(urls, browser_config, 
                                       crawler_config):
    """
    Fetch the contents of multiple web pages concurrently.
    """
    dispatcher = MemoryAdaptiveDispatcher(
        memory_threshold_percent=95.0,
        check_interval=0.5,
        max_session_permit=5,
        # monitor=CrawlerMonitor(
        #      max_visible_rows=15,
        #     display_mode=DisplayMode.DETAILED
        #     )
        )
    async with AsyncWebCrawler() as crawler:
        # Create a dispatcher for concurrent requests
        try:
            startTime = time.time()
            successCnt = 0
            failureCnt = 0
            rets = [] # aggregate results
            results = await crawler.arun_many(
                urls=urls,
                browser_config=browser_config,
                run_config=crawler_config,
                dispatcher=dispatcher
            )
            for result in results:
                if isinstance(result, Exception):
                    print(f"Error fetching {result.url}: {result.error_message}")
                    failureCnt += 1
                    
                elif result.success:
                    print(f"Successfully fetched : {result.url}")
                    print(f"HTML length: {len(result.html)}")
                    print(f"Markdown length: {len(result.markdown)}")
                    print(f"Markdown snippet : {result.markdown[:200]}...")
                    rets.append(result)
                    successCnt += 1
                else:
                    print(f"Failed to fetch {result.url}: {result.error}")
                    failureCnt += 1
            
            print(f"Total URLs: {len(urls)}, Success: {successCnt}, Failures: {failureCnt}")


        except Exception as e:
            print(f"Error during crawling: {e}")
            results = None
        finally:
            await crawler.close()
        return rets 


# Single URL Web scraping test

In [207]:
rst = asyncio.run(getWebPageContent(df.iloc[0].link)) # Get the content of the first URL

[INIT].... → Crawl4AI 0.5.0.post4
[FETCH]... ↓ https://www.hko.gov.hk/textonly/v2/forecast/englis... | Status: True | Time: 0.52s


/home/johnsonhk88/.local/lib/python3.10/site-packages/bs4/builder/_lxml.py:124: DeprecationWarning: The 'strip_cdata' option of HTMLParser() has never done anything and will eventually be removed.
  parser = parser(


[SCRAPE].. ◆ https://www.hko.gov.hk/textonly/v2/forecast/englis... | Time: 0.002s
[COMPLETE] ● https://www.hko.gov.hk/textonly/v2/forecast/englis... | Status: True | Total: 0.53s


In [208]:
(rst[0].model_dump())

{'url': 'https://www.hko.gov.hk/textonly/v2/forecast/englishwx2.htm',
 'html': '<html lang="en"><head>\n\t\t<title>Current Weather</title>\n\t\t<meta name="Keywords" content="Current Weather">\n\t\t<meta name="Description" content="Current Weather">\n\t\t<meta name="Comments" content="WCAG2.0_Verified">\n\t\t<meta http-equiv="Content-Type" content="text/html; CHARSET=UTF-8">\n\t\t<script type="text/javascript" src="/js/jquery/jquery-1.6.4.min.js"></script>\n\t\t<script type="text/javascript" src="/js/clf_textonly.js"></script>\n\t</head>\n\t<body>\n\t<style type="text/css">\n#wcag_logo_area{\n\tdisplay:inline;\n\ttext-align: center;\n}\n#wcag_logo_area a img{\n\tborder:\t0px;\n\tposition:relative;\n\ttop:12px;\n}\npre {\n white-space: pre-wrap;       /* css-3 */\n white-space: -moz-pre-wrap;  /* Mozilla, since 1999 */\n white-space: -pre-wrap;      /* Opera 4-6 */\n white-space: -o-pre-wrap;    /* Opera 7 */\n word-wrap: break-word;       /* Internet Explorer 5.5+ */\n}\n</style>\n<lin

In [209]:
for item in rst:
    print(f"URL: {item.url}")
    # print(f"Title: {item.title}")
    print(f"Content: {item.html[:200]}...")  # Print first 200 characters of content
    # print(f"Images: {item.images}")
    print(f"Links: {item.links}")
    print(f"Markdown: {item.markdown[:200]}...")
    print(f"Metadata: {item.metadata}")  
    # print(f"Text: {item.text[:200]}...")  # Print first 200 characters of text
    print("-" * 80)  # Separator for readability

URL: https://www.hko.gov.hk/textonly/v2/forecast/englishwx2.htm
Content: <html lang="en"><head>
		<title>Current Weather</title>
		<meta name="Keywords" content="Current Weather">
		<meta name="Description" content="Current Weather">
		<meta name="Comments" content="WCAG2....
Links: {'internal': [{'href': 'https://www.hko.gov.hk/textonly/v2/readme.htm', 'text': 'Copyright and Disclaimer', 'title': '', 'base_domain': 'hko.gov.hk'}, {'href': 'https://www.hko.gov.hk/textonly/v2/index.htm', 'text': 'Home', 'title': '', 'base_domain': 'hko.gov.hk'}, {'href': 'https://www.hko.gov.hk/textonly/v2/forecast/chinesewx2.htm', 'text': 'Chinese Version / 中文版本', 'title': '', 'base_domain': 'hko.gov.hk'}], 'external': [{'href': 'http://rss.weather.gov.hk/rss/CurrentWeather.xml', 'text': '', 'title': '', 'base_domain': 'weather.gov.hk'}, {'href': 'http://www.w3.org/WAI/WCAG2AA-Conformance', 'text': '', 'title': '', 'base_domain': 'w3.org'}]}
Markdown: ![Hong Kong Observatory Logo](https://www.hko.gov.h

In [210]:
def extractMarkdown(ret, types="all", extract="raw_markdown"):
    markdowns= []
    if types == "all":
        for item in ret:
            # data = item["_results"][0] # extract # new version
            logger.info(f"Extracting markdown from item: {type(item)}")
            data = item # old version
            print(f"Markdown: {data.markdown.raw_markdown[:200]}...")  
            markdowns.append(data.markdown.raw_markdown)

    elif types == "markdown":
        for item in ret:
            markdowns.append(item["markdown"])
    return markdowns

In [211]:
markdowns = extractMarkdown(rst, types="all", extract="raw_markdown")

2025-06-01 18:56:42,836 - 131742558658560 - 2048757435.py-2048757435:6 - INFO: Extracting markdown from item: <class 'crawl4ai.models.CrawlResult'>


Markdown: ![Hong Kong Observatory Logo](https://www.hko.gov.hk/textonly/images_e/logo_dblue.gif)
# Current Weather
Bulletin updated at 18:02 HKT 01/Jun/2025
```
At 6 p.m. at Hong Kong Observatory :
Air Temperat...


In [212]:
Markdown(markdowns[0])

![Hong Kong Observatory Logo](https://www.hko.gov.hk/textonly/images_e/logo_dblue.gif)
# Current Weather
Bulletin updated at 18:02 HKT 01/Jun/2025
```
At 6 p.m. at Hong Kong Observatory :
Air Temperature : 29 degrees Celsius
Relative Humidity : 81 per cent
Weather Cartoon : No. 76 - Mainly Cloudy 
During the past hour the mean UV Index recorded at King's Park : 0.3
Intensity of UV radiation : low
The air temperatures at other places were:
King's Park           28 degrees;
Wong Chuk Hang         28 degrees;
Ta Kwu Ling           29 degrees;
Lau Fau Shan          30 degrees;
Tai Po             29 degrees;
Sha Tin             28 degrees;
Tuen Mun            29 degrees;
Tseung Kwan O          27 degrees;
Sai Kung            27 degrees;
Cheung Chau           27 degrees;
Chek Lap Kok          29 degrees;
Tsing Yi            29 degrees;
Shek Kong            30 degrees;
Tsuen Wan Ho Koon        28 degrees;
Tsuen Wan Shing Mun Valley   29 degrees;
Hong Kong Park         28 degrees;
Shau Kei Wan          27 degrees;
Kowloon City          27 degrees;
Happy Valley          29 degrees;
Wong Tai Sin          29 degrees;
Stanley             28 degrees;
Kwun Tong            27 degrees;
Sham Shui Po          29 degrees;
Kai Tak Runway Park       27 degrees;
Yuen Long Park         30 degrees;
Tai Mei Tuk           28 degrees.

```
| [Copyright and Disclaimer](https://www.hko.gov.hk/textonly/v2/readme.htm) | [Home](https://www.hko.gov.hk/textonly/v2/index.htm) | [Chinese Version / 中文版本](https://www.hko.gov.hk/textonly/v2/forecast/chinesewx2.htm) | [![Current Weather Report RSS](https://www.hko.gov.hk/img/rss3.gif)](http://rss.weather.gov.hk/rss/CurrentWeather.xml) | 
[![遵守2A級無障礙圖示，萬維網聯盟（W3C）- 無障礙網頁倡議（WAI） Web Content Accessibility Guidelines 2.0](https://www.hko.gov.hk/images/w3c_wcag2.0.gif)](http://www.w3.org/WAI/WCAG2AA-Conformance)


In [213]:
## Test concurrent fetching of multiple web pages

In [215]:
res2 = asyncio.run(getWebPageContentsConcurrent(df.link.tolist(), 
                                                browser_config=browser_config, 
                                                crawler_config=crawler_config1))

[INIT].... → Crawl4AI 0.5.0.post4
[FETCH]... ↓ https://www.hko.gov.hk/textonly/v2/forecast/englis... | Status: True | Time: 0.75s
[SCRAPE].. ◆ https://www.hko.gov.hk/textonly/v2/forecast/englis... | Time: 0.001s
[COMPLETE] ● https://www.hko.gov.hk/textonly/v2/forecast/englis... | Status: True | Total: 0.76s
[FETCH]... ↓ https://www.hko.gov.hk/textonly/forecast/englishwx... | Status: True | Time: 0.75s
[SCRAPE].. ◆ https://www.hko.gov.hk/textonly/forecast/englishwx... | Time: 0.001s
[COMPLETE] ● https://www.hko.gov.hk/textonly/forecast/englishwx... | Status: True | Total: 0.76s
[FETCH]... ↓ https://www.chinahighlights.com/hong-kong/weather.... | Status: True | Time: 0.76s
[SCRAPE].. ◆ https://www.chinahighlights.com/hong-kong/weather.... | Time: 0.033s
[COMPLETE] ● https://www.chinahighlights.com/hong-kong/weather.... | Status: True | Total: 0.80s
[FETCH]... ↓ https://www.thestandard.com.hk/hong-kong-news/arti... | Status: True | Time: 0.99s
[SCRAPE].. ◆ https://www.thestandard.com.hk/h

In [216]:
res2

[CrawlResultContainer([CrawlResult(url='https://www.hko.gov.hk/textonly/v2/forecast/englishwx2.htm', html='<html lang="en"><head>\n\t\t<title>Current Weather</title>\n\t\t<meta name="Keywords" content="Current Weather">\n\t\t<meta name="Description" content="Current Weather">\n\t\t<meta name="Comments" content="WCAG2.0_Verified">\n\t\t<meta http-equiv="Content-Type" content="text/html; CHARSET=UTF-8">\n\t\t<script type="text/javascript" src="/js/jquery/jquery-1.6.4.min.js"></script>\n\t\t<script type="text/javascript" src="/js/clf_textonly.js"></script>\n\t</head>\n\t<body>\n\t<style type="text/css">\n#wcag_logo_area{\n\tdisplay:inline;\n\ttext-align: center;\n}\n#wcag_logo_area a img{\n\tborder:\t0px;\n\tposition:relative;\n\ttop:12px;\n}\npre {\n white-space: pre-wrap;       /* css-3 */\n white-space: -moz-pre-wrap;  /* Mozilla, since 1999 */\n white-space: -pre-wrap;      /* Opera 4-6 */\n white-space: -o-pre-wrap;    /* Opera 7 */\n word-wrap: break-word;       /* Internet Explorer

In [217]:
markdowns2 = extractMarkdown(res2, types="all", extract="raw_markdown")

2025-06-01 18:57:51,187 - 131742558658560 - 2048757435.py-2048757435:6 - INFO: Extracting markdown from item: <class 'crawl4ai.async_webcrawler.CrawlResultContainer'>


Markdown: ![Hong Kong Observatory Logo](https://www.hko.gov.hk/textonly/images_e/logo_dblue.gif)
# Current Weather
Bulletin updated at 18:02 HKT 01/Jun/2025
```
At 6 p.m. at Hong Kong Observatory :
Air Temperat...


2025-06-01 18:57:51,190 - 131742558658560 - 2048757435.py-2048757435:6 - INFO: Extracting markdown from item: <class 'crawl4ai.async_webcrawler.CrawlResultContainer'>


Markdown: ![Hong Kong Observatory Logo](https://www.hko.gov.hk/images_e/logo_dblue.gif)
# Current Weather
_Bulletin updated at 18:02 HKT 01/Jun/2025_
```
At 6 p.m. at Hong Kong Observatory :
Air Temperature : 2...


2025-06-01 18:57:51,192 - 131742558658560 - 2048757435.py-2048757435:6 - INFO: Extracting markdown from item: <class 'crawl4ai.async_webcrawler.CrawlResultContainer'>


Markdown:   * [ ![chinahighlights logo](https://data.chinahighlights.com/pic/logo/china-highlights-top-navigation-logo.png)](https://www.chinahighlights.com/)
  * [China Tours](https://www.chinahighlights.com/t...


2025-06-01 18:57:51,194 - 131742558658560 - 2048757435.py-2048757435:6 - INFO: Extracting markdown from item: <class 'crawl4ai.async_webcrawler.CrawlResultContainer'>


Markdown: [![logo](https://www.thestandard.com.hk/_next/static/media/TS_logo_116x64.cb1de3b7.png)](https://www.thestandard.com.hk/)
[News](https://www.thestandard.com.hk/news)[Comment](https://www.thestandard.c...


2025-06-01 18:57:51,196 - 131742558658560 - 2048757435.py-2048757435:6 - INFO: Extracting markdown from item: <class 'crawl4ai.async_webcrawler.CrawlResultContainer'>


Markdown: [ ![weather25.com](https://www.weather25.com/images/weather25-black.svg) ](https://www.weather25.com "weather25.com")
[United States](https://www.weather25.com/north-america/usa "Weather in United Sta...
